In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

# splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 50,)

splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 600,
    chunk_overlap = 100,
)

loader = UnstructuredFileLoader("./files/1984_chap_one.docx")

# docs = loader.load()
# splitter.split_documents(docs)

result = loader.load_and_split(text_splitter=splitter )
print(len(result))

Created a chunk of size 964, which is longer than the specified 600
Created a chunk of size 775, which is longer than the specified 600
Created a chunk of size 955, which is longer than the specified 600
Created a chunk of size 923, which is longer than the specified 600
Created a chunk of size 1169, which is longer than the specified 600
Created a chunk of size 822, which is longer than the specified 600
Created a chunk of size 701, which is longer than the specified 600
Created a chunk of size 746, which is longer than the specified 600
Created a chunk of size 736, which is longer than the specified 600
Created a chunk of size 1111, which is longer than the specified 600
Created a chunk of size 992, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 1742, which is longer than the specified 600
Created a chunk of size 2002, which is longer than the specified 600
Created a chunk of size 1901, which is longe

39
